In [2]:
import tkinter as tk
from tkinter import *
import numpy as np
import copy
import random

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.losses import MSE
from tensorflow.keras.optimizers import Adam

from collections import deque, namedtuple
from gym import Env
from gym.spaces import Discrete, MultiDiscrete

import time
import random

C:\Users\parij\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# 1. Rubik's Cube GUI using Tkinter

In [59]:
COLOR_KEY = {
    0 : 'orange',
    1 : 'green',
    2 : 'white',
    3 : 'blue',
    4 : 'yellow',
    5 : 'red'
}


class RubiksCubeGUI:
    def __init__(self, isDesign=True, cube=None):
        self.root = Tk()
        self.cube_colors = np.array([[[0] * 3 for _ in range(3)], 
                                     [[1] * 3 for _ in range(3)],
                                     [[2] * 3 for _ in range(3)],
                                     [[3] * 3 for _ in range(3)],
                                     [[4] * 3 for _ in range(3)],
                                     [[5] * 3 for _ in range(3)]])
        
        self.isDesign = isDesign
        if cube is not None:
            self.cube = cube.reshape((6, 3, 3))
        else:
            self.cube = np.copy(self.cube_colors)
            
        self.actions = { 
            0: self.top,
            1: lambda: self.top(clockwise=False),
            2: self.bottom,
            3: lambda: self.bottom(clockwise=False),
            4: self.left,
            5: lambda: self.left(clockwise=False),
            6: self.right,
            7: lambda: self.right(clockwise=False),
            8: self.front,
            9: lambda: self.front(clockwise=False),
            10: self.back,
            11: lambda: self.back(clockwise=False),
        }
        
        self.cur_colour = 2
        self.cube_grid = []
    
    def set_colour(self, c):
        self.cur_colour = c

    def change_color(self, face, row, col):
        self.cube[face][row][col] = self.cur_colour
        self.cube_grid[face][row][col].config(bg=COLOR_KEY[self.cube[face][row][col]])
        
    def shuffleBoard(self):
        self.shuffle()
        for face in range(6):
            for row in range(3):
                for col in range(3):
                    self.cube_grid[face][row][col].config(bg=COLOR_KEY[self.cube[face][row][col]])
        
    def saveBoard(self):
        noc = [0, 0, 0, 0, 0, 0]
        for face in range(6):
            for row in range(3):
                for col in range(3):
                    noc[self.cube[face][row][col]] += 1
                    
        if noc == [9, 9, 9, 9, 9, 9]:
            self.root.destroy()
        
    def designCube(self):
        frame = Frame(self.root)
        frame.pack()
        bottomframe = Frame(frame)
        bottomframe.pack(side=BOTTOM)

        redbutton = Button(frame, text='Red', fg='red', bg='black', command=lambda: self.set_colour(5))
        redbutton.pack(side=LEFT)

        greenbutton = Button(frame, text='Green', fg='green', bg='black', command=lambda: self.set_colour(1))
        greenbutton.pack(side=LEFT)

        bluebutton = Button(frame, text='Blue', fg='blue', bg='black', command=lambda: self.set_colour(3))
        bluebutton.pack(side=LEFT)

        yellowbutton = Button(frame, text='Yellow', fg='yellow', bg='black', command=lambda: self.set_colour(4))
        yellowbutton.pack(side=LEFT)

        whitebutton = Button(frame, text='White', fg='white', bg='black', command=lambda: self.set_colour(2))
        whitebutton.pack(side=LEFT)

        orangebutton = Button(frame, text='Orange', fg='orange', bg='black', command=lambda: self.set_colour(0))
        orangebutton.pack(side=LEFT)
        
        shufflebutton = Button(bottomframe, text='SHUFFLE', fg='black', bg='white', command=lambda: self.shuffleBoard())
        shufflebutton.pack(side=LEFT)
        
        savebutton = Button(bottomframe, text='SAVE', fg='black', bg='white', command=lambda: self.saveBoard())
        savebutton.pack(side=LEFT)
        
        # Create a frame for the grid squares
        grid_frames = []
        main_frame = tk.Frame(self.root)
        main_frame.pack(padx=10, pady=10)

        for i in range(6):
            grid_frame = tk.Frame(main_frame)
            if i == 0:
                grid_frame.grid(row=0, column=1, padx=5, pady=5)
            elif i == 5:
                grid_frame.grid(row=2, column=1, padx=5, pady=5)
            else:
                grid_frame.grid(row=1, column=i-1, padx=5, pady=5)

            grid_frames.append(grid_frame)
            # Creating the grids
            squares = []

            for row in range(3):
                square_row = []
                for col in range(3):
                    square = tk.Frame(grid_frames[i], width=50, height=50)
                    square.grid(row=row, column=col, padx=1, pady=1)
                    # Associate a button with each square
                    button = tk.Button(square, height=3, width=5, bg=COLOR_KEY[self.cube[i][row][col]])
                    button.pack()
                    if self.isDesign and (row != 1 or col != 1):
                        button.config(command=lambda r=row, c=col, face=i: self.change_color(face, r, c))
                    square_row.append(button)

                squares.append(square_row)
            self.cube_grid.append(squares)

        self.root.mainloop()
        
    def rotate_face_clockwise(self, face):
        self.cube[face] = np.rot90(self.cube[face], 3)

    def rotate_face_counterclockwise(self, face):
        self.cube[face] = np.rot90(self.cube[face])

    def top(self, clockwise=True):
        if clockwise:
            self.rotate_face_clockwise(0)
            temp = np.copy(self.cube[1, 0, :])
            self.cube[1, 0, :] = self.cube[2, 0, :]
            self.cube[2, 0, :] = self.cube[3, 0, :]
            self.cube[3, 0, :] = self.cube[4, 0, :]
            self.cube[4, 0, :] = temp  
        else:
            self.rotate_face_counterclockwise(0)
            temp = np.copy(self.cube[1, 0, :])
            self.cube[1, 0, :] = self.cube[4, 0, :]
            self.cube[4, 0, :] = self.cube[3, 0, :]
            self.cube[3, 0, :] = self.cube[2, 0, :]
            self.cube[2, 0, :] = temp
        return self.cube

    def bottom(self, clockwise=True):
        if clockwise:
            self.rotate_face_counterclockwise(5)
            temp = np.copy(self.cube[1, 2, :])
            self.cube[1, 2, :] = self.cube[2, 2, :]
            self.cube[2, 2, :] = self.cube[3, 2, :]
            self.cube[3, 2, :] = self.cube[4, 2, :]
            self.cube[4, 2, :] = temp
        else:
            self.rotate_face_clockwise(5)
            temp = np.copy(self.cube[1, 2, :])
            self.cube[1, 2, :] = self.cube[4, 2, :]
            self.cube[4, 2, :] = self.cube[3, 2, :]
            self.cube[3, 2, :] = self.cube[2, 2, :]
            self.cube[2, 2, :] = temp
        return self.cube

    def left(self, clockwise=True):
        if clockwise:
            self.rotate_face_clockwise(1)
            temp = np.copy(self.cube[0, :, 0])
            self.cube[0, :, 0] = self.cube[4, ::-1, 2]
            self.cube[4, :, 2] = self.cube[5, ::-1, 0]
            self.cube[5, :, 0] = self.cube[2, :, 0]
            self.cube[2, :, 0] = temp
        else:
            self.rotate_face_counterclockwise(1)
            temp = np.copy(self.cube[0, :, 0])
            self.cube[0, :, 0] = self.cube[2, :, 0]
            self.cube[2, :, 0] = self.cube[5, :, 0]
            self.cube[5, :, 0] = self.cube[4, ::-1, 2]
            self.cube[4, :, 2] = temp[::-1]
        return self.cube

    def right(self, clockwise=True):
        if clockwise:
            self.rotate_face_clockwise(3)
            temp = np.copy(self.cube[0, :, 2])
            self.cube[0, :, 2] = self.cube[2, :, 2]
            self.cube[2, :, 2] = self.cube[5, :, 2]
            self.cube[5, :, 2] = self.cube[4, ::-1, 0]
            self.cube[4, :, 0] = temp[::-1]
        else:
            self.rotate_face_counterclockwise(3)
            temp = np.copy(self.cube[0, :, 2])
            self.cube[0, :, 2] = self.cube[4, ::-1, 0]
            self.cube[4, :, 0] = self.cube[5, ::-1, 2]
            self.cube[5, ::-1, 2] = self.cube[2, :, 2]
            self.cube[2, :, 2] = temp
        return self.cube

    def front(self, clockwise=True):
        if clockwise:
            self.rotate_face_clockwise(2)
            temp = np.copy(self.cube[0, 2, :])
            self.cube[0, 2, :] = self.cube[1, ::-1, 2]
            self.cube[1, :, 2] = self.cube[5, 0, :]
            self.cube[5, 0, :] = self.cube[3, ::-1, 0]
            self.cube[3, :, 0] = temp
        else:
            self.rotate_face_counterclockwise(2)
            temp = np.copy(self.cube[0, 2, :])
            self.cube[0, 2, :] = self.cube[3, :, 0]
            self.cube[3, :, 0] = self.cube[5, 0, ::-1]
            self.cube[5, 0, :] = self.cube[1, :, 2]
            self.cube[1, :, 2] = temp[::-1]
        return self.cube

    def back(self, clockwise=True):
        if clockwise:
            self.rotate_face_clockwise(4)
            temp = np.copy(self.cube[0, 0, :])
            self.cube[0, 0, :] = self.cube[3, :, 2]
            self.cube[3, :, 2] = self.cube[5, 2, ::-1]
            self.cube[5, 2, :] = self.cube[1, :, 0]
            self.cube[1, :, 0] = temp[::-1]
        else:
            self.rotate_face_counterclockwise(4)
            temp = np.copy(self.cube[0, 0, :])
            self.cube[0, 0, :] = self.cube[1, ::-1, 0]
            self.cube[1, :, 0] = self.cube[5, 2, :]
            self.cube[5, 2, :] = self.cube[3, ::-1, 2]
            self.cube[3, :, 2] = temp
        return self.cube    
    
    def shuffle(self):
        n = random.randint(5, 15)
        for i in range(n):
            r = random.randint(0, 5)
            self.actions[2 * r]()
        
        return self.cube

# 2. Reinforcement Learning Environment

In [60]:
# Values of the constants
cube_number = 100

In [61]:
class CubeEnv (Env):
    def __init__(self):
        # Moves that we can make
        self.action_space = Discrete(12)
        # All possible combinations of a rubiks cube
        self.observation_space = MultiDiscrete([6]*54)
        
        #instantiating RubiksCubeGUI class
        self.myCube = RubiksCubeGUI()
        self.myCube.designCube()
        # Use the cube state obtained from the RubiksCubeGUI class
        self.state = self.myCube.cube.reshape(54,)
        # Maximum no. of moves the agent is allowed to make
        self.moves_length = cube_number
        self.root = Tk()
        self.cube_grid = []
        
        grid_frames = []
        main_frame = tk.Frame(self.root)
        main_frame.pack(padx=10, pady=10)

        for i in range(6):
            grid_frame = tk.Frame(main_frame)
            if i == 0:
                grid_frame.grid(row=0, column=1, padx=5, pady=5)
            elif i == 5:
                grid_frame.grid(row=2, column=1, padx=5, pady=5)
            else:
                grid_frame.grid(row=1, column=i-1, padx=5, pady=5)

            grid_frames.append(grid_frame)
            # Creating the grids
            squares = []

            for row in range(3):
                square_row = []
                for col in range(3):
                    square = tk.Frame(grid_frames[i], width=50, height=50, bg=COLOR_KEY[self.myCube.cube[i][row][col]])
                    square.grid(row=row, column=col, padx=1, pady=1)
                    square_row.append(square)
                squares.append(square_row)
            self.cube_grid.append(squares)

        
    
    def step(self, action):
        # Applying the action
        self.state = self.myCube.actions[action]().reshape(54,)
        self.myCube.cube = self.state.reshape(6, 3, 3)
        # Reduce no. of moves left by 1
        self.moves_length -= 1
        
        # Calculating the reward
        if (self.state.reshape(6, 3, 3) == self.myCube.cube_colors).all():
            reward = 100
            done = True
        else:
            reward = 0
            done = False
            
#         # Check if no. of moves exceeded maximum number of moves
#         if  (self.state == self.myCube.cube_colors).all():
#             done = True
#         else:
#             done = False
        if self.moves_length <= 0: done = True
        
        info = {}
        
        return self.state, reward, done, info
    
    def render(self):
        state3d = self.state.reshape(6, 3, 3)
        for face in range(6):
            for row in range(3):
                for col in range(3):
                    self.cube_grid[face][row][col].configure(bg=COLOR_KEY[state3d[face][row][col]])
        
    def reset(self):
        self.myCube.cube = np.copy(self.myCube.cube_colors)
        self.state = self.myCube.shuffle().reshape(54,)
        self.moves_length = cube_number
        return self.state
    

In [62]:
env = CubeEnv()
env.reset()
env.state.shape

(54,)

In [63]:
for episode in range(10):
    state = env.reset()
    done = False
    score = 0
    i = 0
    
    while not done and i < 30:
        i += 1
        action = env.action_space.sample()
        n_state, reward, done, _ = env.step(action)
        score += reward
    print(f'Episode: {episode + 1} Score: {score}')
    
env.root.mainloop()

Episode: 1 Score: 0
Episode: 2 Score: 0
Episode: 3 Score: 0
Episode: 4 Score: 0
Episode: 5 Score: 0
Episode: 6 Score: 0
Episode: 7 Score: 0
Episode: 8 Score: 0
Episode: 9 Score: 0
Episode: 10 Score: 0


# 3.0 PPO

In [55]:
from stable_baselines3 import PPO

model = PPO('MlpPolicy', env, verbose=1)

model.learn(total_timesteps=1000000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 6.14     |
|    ep_rew_mean     | 96       |
| time/              |          |
|    fps             | 388      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 10.9         |
|    ep_rew_mean          | 92           |
| time/                   |              |
|    fps                  | 278          |
|    iterations           | 2            |
|    time_elapsed         | 14           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0029897685 |
|    clip_fraction        | 0.0349       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.685       |
|    explained_variance   | 0.876        |
|    learning_r

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 7.04       |
|    ep_rew_mean          | 95         |
| time/                   |            |
|    fps                  | 247        |
|    iterations           | 11         |
|    time_elapsed         | 91         |
|    total_timesteps      | 22528      |
| train/                  |            |
|    approx_kl            | 0.01146033 |
|    clip_fraction        | 0.0928     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.618     |
|    explained_variance   | 0.95       |
|    learning_rate        | 0.0003     |
|    loss                 | 16.2       |
|    n_updates            | 1080       |
|    policy_gradient_loss | -0.00201   |
|    value_loss           | 43.7       |
----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.08        |
|    ep_rew_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 5           |
|    ep_rew_mean          | 97          |
| time/                   |             |
|    fps                  | 244         |
|    iterations           | 21          |
|    time_elapsed         | 175         |
|    total_timesteps      | 43008       |
| train/                  |             |
|    approx_kl            | 0.013511406 |
|    clip_fraction        | 0.0927      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.536      |
|    explained_variance   | 0.983       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.956       |
|    n_updates            | 1180        |
|    policy_gradient_loss | -0.0122     |
|    value_loss           | 24.3        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.04  

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 7.94       |
|    ep_rew_mean          | 94         |
| time/                   |            |
|    fps                  | 243        |
|    iterations           | 31         |
|    time_elapsed         | 260        |
|    total_timesteps      | 63488      |
| train/                  |            |
|    approx_kl            | 0.02751098 |
|    clip_fraction        | 0.116      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.461     |
|    explained_variance   | 0.983      |
|    learning_rate        | 0.0003     |
|    loss                 | 14.1       |
|    n_updates            | 1280       |
|    policy_gradient_loss | -0.0109    |
|    value_loss           | 31.6       |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 4.02       |
|    ep_rew_mean

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 4.08        |
|    ep_rew_mean          | 98          |
| time/                   |             |
|    fps                  | 244         |
|    iterations           | 41          |
|    time_elapsed         | 344         |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.030803446 |
|    clip_fraction        | 0.166       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.504      |
|    explained_variance   | 0.995       |
|    learning_rate        | 0.0003      |
|    loss                 | 0.0624      |
|    n_updates            | 1380        |
|    policy_gradient_loss | -0.0167     |
|    value_loss           | 9.51        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 6.02  

...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.16        |
|    ep_rew_mean          | 100         |
| time/                   |             |
|    fps                  | 197         |
|    iterations           | 441         |
|    time_elapsed         | 4579        |
|    total_timesteps      | 903168      |
| train/                  |             |
|    approx_kl            | 0.004410336 |
|    clip_fraction        | 0.0512      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.27       |
|    explained_variance   | 0.585       |
|    learning_rate        | 0.0003      |
|    loss                 | 34.4        |
|    n_updates            | 5380        |
|    policy_gradient_loss | -0.00379    |
|    value_loss           | 119         |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.12    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 3.7         |
|    ep_rew_mean          | 99          |
| time/                   |             |
|    fps                  | 198         |
|    iterations           | 451         |
|    time_elapsed         | 4659        |
|    total_timesteps      | 923648      |
| train/                  |             |
|    approx_kl            | 0.012123969 |
|    clip_fraction        | 0.163       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.373      |
|    explained_variance   | 0.981       |
|    learning_rate        | 0.0003      |
|    loss                 | 5.11        |
|    n_updates            | 5480        |
|    policy_gradient_loss | -0.0305     |
|    value_loss           | 26          |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2.36  

----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 2.18       |
|    ep_rew_mean          | 100        |
| time/                   |            |
|    fps                  | 199        |
|    iterations           | 461        |
|    time_elapsed         | 4739       |
|    total_timesteps      | 944128     |
| train/                  |            |
|    approx_kl            | 0.03286409 |
|    clip_fraction        | 0.067      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.105     |
|    explained_variance   | 0.898      |
|    learning_rate        | 0.0003     |
|    loss                 | 2.96       |
|    n_updates            | 5580       |
|    policy_gradient_loss | -0.023     |
|    value_loss           | 9.54       |
----------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 21.7      |
|    ep_rew_mean   

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 23.7        |
|    ep_rew_mean          | 82          |
| time/                   |             |
|    fps                  | 200         |
|    iterations           | 471         |
|    time_elapsed         | 4818        |
|    total_timesteps      | 964608      |
| train/                  |             |
|    approx_kl            | 0.004298671 |
|    clip_fraction        | 0.0379      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.269      |
|    explained_variance   | 0.841       |
|    learning_rate        | 0.0003      |
|    loss                 | 26.3        |
|    n_updates            | 5680        |
|    policy_gradient_loss | -0.00983    |
|    value_loss           | 79.2        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 26.3

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 7.32        |
|    ep_rew_mean          | 95          |
| time/                   |             |
|    fps                  | 201         |
|    iterations           | 481         |
|    time_elapsed         | 4896        |
|    total_timesteps      | 985088      |
| train/                  |             |
|    approx_kl            | 0.006028797 |
|    clip_fraction        | 0.0757      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.381      |
|    explained_variance   | 0.899       |
|    learning_rate        | 0.0003      |
|    loss                 | 40.1        |
|    n_updates            | 5780        |
|    policy_gradient_loss | -0.00931    |
|    value_loss           | 119         |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 4.4   

In [58]:
model.save('Rubiks_Cube_solver')

In [65]:
from stable_baselines3.common.evaluation import evaluate_policy

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"Mean reward: {mean_reward} +/- {std_reward:.2f}")

Mean reward: 0.0 +/- 0.00
